In [18]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# Load dataset
max_features = 20000  # Number of words to consider as features
maxlen = 80  # Cut texts after this number of words

# Load IMDB data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Preprocess data: pad sequences to ensure each input sequence has the same length
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)


Divide Data into Training and Testing sets


In [19]:
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence

# Load dataset
max_features = 20000  # Number of words to consider as features
maxlen = 80  # Cut texts after this number of words

(X, y), (_, _) = imdb.load_data(num_words=max_features)  # Load full dataset

# Preprocess data: pad sequences to ensure each input sequence has the same length
X = sequence.pad_sequences(X, maxlen=maxlen)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now you can proceed with model training and evaluation
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")


Training set size: 20000
Testing set size: 5000


Build the Gated Recurrent Units(GRU)Model


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))  # Embedding layer for word embeddings
model.add(GRU(128, return_sequences=False))  # GRU layer with 128 units
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Training the GRU Model


In [16]:
# Train the GRU model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)



Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 80s 154ms/step - accuracy: 0.6816 - loss: 0.5708 - val_accuracy: 0.8335 - val_loss: 0.3761
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 80s 151ms/step - accuracy: 0.9015 - loss: 0.2508 - val_accuracy: 0.8248 - val_loss: 0.3888
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 80s 148ms/step - accuracy: 0.9592 - loss: 0.1198 - val_accuracy: 0.8223 - val_loss: 0.5056
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 80s 144ms/step - accuracy: 0.9868 - loss: 0.0435 - val_accuracy: 0.8240 - val_loss: 0.6041
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 79s 139ms/step - accuracy: 0.9924 - loss: 0.0260 - val_accuracy: 0.8220 - val_loss: 0.8589


Text Generation using the Trained Model


In [24]:
import random
import numpy as np
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming reverse_word_map is a dictionary mapping word indices back to words

# Function to generate text
def generate_text(seed_text, model, max_sequence_len, reverse_word_map):
    for _ in range(50):  # Number of words to generate
        token_list = text_to_word_sequence(seed_text)  # Tokenize seed text
        token_list = [reverse_word_map.get(word, 0) for word in token_list]  # Convert words to indices
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')  # Pad sequences
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)

        # Handle KeyError by using <UNK> for unknown words
        next_word = reverse_word_map.get(predicted[0], "<UNK>")

        seed_text += " " + next_word  # Append predicted word to seed text
    return seed_text

# Example usage of generate_text
word_index = imdb.get_word_index()  # Original word to index mapping from IMDB dataset
reverse_word_map = {index: word for word, index in word_index.items()}  # Reverse map

# Ensure the special tokens exist in reverse_word_map
reverse_word_map[0] = "<PAD>"  # Padding token
reverse_word_map[1] = "<START>"  # Start token (depends on dataset)
reverse_word_map[2] = "<UNK>"  # Unknown token

# Assuming 'model' and 'maxlen' are already defined and trained
generated_text = generate_text("This movie", model, maxlen, reverse_word_map)
print(generated_text)


This movie <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


Evaluate Models's Accuracy


In [25]:
# Evaluate model on test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.5119 - loss: 0.6929
Test Accuracy: 51.98%


Compare accuracy of Long sort term memory and Gated recurrent Unit models for text generation using data from tensorflow.keras.datasets.


Load and Preprocess the Data

In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# Load dataset
max_features = 20000  # Number of words to consider as features
maxlen = 80  # Cut texts after this number of words

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Preprocess data: pad sequences to ensure each input sequence has the same length
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)


Build the LSTM Model


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Build the LSTM model
def create_lstm_model():
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length=maxlen))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

lstm_model = create_lstm_model()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Build the GRU Model


In [32]:
from tensorflow.keras.layers import GRU

# Build the GRU model
def create_gru_model():
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length=maxlen))
    model.add(GRU(128, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

gru_model = create_gru_model()



Train Both Models

Train with LSTM model

In [31]:
# Train the LSTM model
lstm_history = lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 106s 170ms/step - accuracy: 0.9900 - loss: 0.0361 - val_accuracy: 0.8072 - val_loss: 0.6997
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 141s 168ms/step - accuracy: 0.9911 - loss: 0.0290 - val_accuracy: 0.8156 - val_loss: 0.7399
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 141s 166ms/step - accuracy: 0.9958 - loss: 0.0144 - val_accuracy: 0.7966 - val_loss: 1.0219
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 141s 164ms/step - accuracy: 0.9972 - loss: 0.0129 - val_accuracy: 0.8206 - val_loss: 1.0940
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 164ms/step - accuracy: 0.9957 - loss: 0.0139 - val_accuracy: 0.8176 - val_loss: 1.0708


Train with GRU model

In [33]:
# Train the GRU model
gru_history = gru_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 93s 143ms/step - accuracy: 0.7039 - loss: 0.5341 - val_accuracy: 0.8246 - val_loss: 0.3833
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 139s 138ms/step - accuracy: 0.9113 - loss: 0.2300 - val_accuracy: 0.8370 - val_loss: 0.4038
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 145s 143ms/step - accuracy: 0.9587 - loss: 0.1176 - val_accuracy: 0.8310 - val_loss: 0.4583
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 136ms/step - accuracy: 0.9822 - loss: 0.0533 - val_accuracy: 0.8236 - val_loss: 0.5626
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 138ms/step - accuracy: 0.9915 - loss: 0.0288 - val_accuracy: 0.8122 - val_loss: 0.7448


Evaluate and Compare Accuracy


In [30]:
# Evaluate LSTM model
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)
print(f"LSTM Test Accuracy: {lstm_accuracy * 100:.2f}%")

# Evaluate GRU model
gru_loss, gru_accuracy = gru_model.evaluate(X_test, y_test)
print(f"GRU Test Accuracy: {gru_accuracy * 100:.2f}%")

# Compare the accuracies
if lstm_accuracy > gru_accuracy:
    print("LSTM performed better.")
else:
    print("GRU performed better.")


782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 48ms/step - accuracy: 0.7982 - loss: 0.7346
LSTM Test Accuracy: 80.11%
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.8197 - loss: 0.7604
GRU Test Accuracy: 82.22%
GRU performed better.
